## Example: The Tiger Problem as a Markov Decision Problem (MDP)

<center>
    <img src="figs/Fig-Linear-MDP-Schematic.png" style="align:right; width:80%">
</center>

An agent trapped in a long hallway with two doors at either end. Behind the red door is a tiger (and certain death), while behind the green door is freedom. If the agent opens the red door, the agent is eaten (and receives a large negative reward). However, if the agent opens the green door, it escapes and gets a positive reward. 

For this problem, the MDP has the tuple components:
* $\mathcal{S} = \left\{1,2,\dots,N\right\}$ while the action set is $\mathcal{A} = \left\{a_{1},a_{2}\right\}$; action $a_{1}$ moves the agent one state to the right, action $a_{2}$ moves the agent one state to the left.
* The agent receives a reward of +10 for entering state 1 (escapes). However, the agent is penalized -100 for entering state N (eaten by the tiger).  Finally, the agent is not charged to move to adjacent locations.
* Let the probability of correctly executing the action $a_{j}$ be $\alpha$

Let's compute $U^{\pi}(s)$ for different choices for the policy function $\pi$.

## Setup
Let's load some packages that are required for the example by calling the `include(...)` function on our initialization file `Include.jl`:

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-13/L13c`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-13/L13c/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-13/L13c/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-13/L13c/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-13/L13c/Manifest.toml`


In [2]:
# setup some global constants -
α = 0.75; # probability of moving the direction we are expect
γ = 0.95; # discount factor

## Setup the states, actions, transitions and rewards
Let's set up the states $\mathcal{S}$ and actions $\mathcal{A}$ sets: 
* We have `safety` at index `1`, but the `tiger` lives at index `10`. Thus, the states set $\mathcal{S} = \left\{1,\dotsc,10\right\}$
* We have `3` possible actions, `move left,` `move right` or `stand still` in action set $\mathcal{A} = \left\{1,2,3\right\}$

In [3]:
# setup the states and actions -
safety = 1;
tiger = 10;

# Setup the states -
states = range(safety,stop=tiger, step=1) |> collect;

# Setup the actions
actions = [1,2,3]; # a₁ = move left, a₂ = move right, a₃ = stand still

### Setup the rewards array

The rewards $\mathbf{R}$ is a $\dim\mathcal{S}\times\dim\mathcal{A}$ array whose $R(s,a)$ element holds the reward for performing action $a$ in state $s$:

In [4]:
# setup the rewards -
R = Array{Float64,2}(undef,length(states), length(actions));
fill!(R,0.0) # fill R 0.0
R[safety + 1, 1] = 10; # reward for entering escape door
R[tiger - 1, 2] = -100; # reward for opening door with the tiger 
R[2:end-1, 3] .= -1; # reward for doing doing

In [5]:
R

10×3 Matrix{Float64}:
  0.0     0.0   0.0
 10.0     0.0  -1.0
  0.0     0.0  -1.0
  0.0     0.0  -1.0
  0.0     0.0  -1.0
  0.0     0.0  -1.0
  0.0     0.0  -1.0
  0.0     0.0  -1.0
  0.0  -100.0  -1.0
  0.0     0.0   0.0

### Setup the transitions array
The transition probability array $\mathbf{T}(s^{\prime}\,|\,s,a)$ is a $\dim\mathcal{S}\times\dim\mathcal{S}\times\dim\mathcal{A}$ [multi-dimensional array](https://docs.julialang.org/en/v1/manual/arrays/#man-multi-dim-arrays) that encodes the physics of the world, and what happens if we take action $a\in\mathcal{A}$ 
* For an action $a\in\mathcal{A}$, the array $\mathbf{T}(s^{\prime}\,|\,s,a)$ (which we sometimes denote as $\mathbf{T}_{a}$) is a $\dim\mathcal{S}\times\dim\mathcal{S}$ array encoding states $s$ on the rows, and states $s^{\prime}$ on the columns. Because the entries are probabilities, the rows of $\mathbf{T}_{a}$ must sum to `1`.

In [6]:
# Setup the transitions
T = Array{Float64,3}(undef, length(states), length(states), length(actions));
fill!(T,0.0);

# We need to put values into the transition array (these are probabilities, so eah row much sum to 1)
T[safety, 1, 1:length(actions)] .= 1.0; # if we are in state 1, we stay in state 1 ∀a ∈ 𝒜
T[tiger, tiger, 1:length(actions)] .= 1.0; # if we are in state 5, we stay in state 5 

#### Left, Right, and Listen to Actions
We encode the probability of reaching the next state $s^{\prime}\leftarrow(s, a)$ in the entries of $\mathbf{T}_{a}$. In response to action $a\in\mathcal{A}$, there is a probability of $\alpha$ that we end up in the correct next state, but a $(1-\alpha)$ that we end up in an incorrect state. 

`Unhide` the block below to see how we populate the transition probability array $\mathbf{T}(s^{\prime}\,|\,s,a)$:

In [7]:
# left actions -
for s ∈ 2:(tiger - 1)
    T[s,s-1,1] = α;
    T[s,s+1,1] = (1-α);
end

# right actions -
for s ∈ 2:(tiger - 1)
    T[s,s-1,2] = (1-α);
    T[s,s+1,2] = α; 
end

# listen action (we don't move to a new state)
for s ∈ 2:(tiger-1)
    T[s,s,3] = 1.0;
end

In [8]:
T[:,:,1] # probability matrix for taking action aᵢ

10×10 Matrix{Float64}:
 1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
 0.75  0.0   0.25  0.0   0.0   0.0   0.0   0.0   0.0   0.0
 0.0   0.75  0.0   0.25  0.0   0.0   0.0   0.0   0.0   0.0
 0.0   0.0   0.75  0.0   0.25  0.0   0.0   0.0   0.0   0.0
 0.0   0.0   0.0   0.75  0.0   0.25  0.0   0.0   0.0   0.0
 0.0   0.0   0.0   0.0   0.75  0.0   0.25  0.0   0.0   0.0
 0.0   0.0   0.0   0.0   0.0   0.75  0.0   0.25  0.0   0.0
 0.0   0.0   0.0   0.0   0.0   0.0   0.75  0.0   0.25  0.0
 0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.75  0.0   0.25
 0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0

## Build the MDP problem object and estimate the utility $U^{\pi}$ 
Construct a `MyMDPProblemModel` instance with values associated with the problem. 
* We must pass the states `𝒮,` the actions `𝒜,` the transition matrix `T,` the reward matrix `R,` and the discount factor `γ` into the `build(...)` method. We store the MDP model in the `m` variable:

In [9]:
m = build(MyMDPProblemModel, 
    (𝒮 = states, 𝒜 = actions, T = T, R = R, γ = γ));

Next, let's formulate some simple policies for this problem, `always_move_right,` `always_move_left`, and `always_listen.` We'll test how good or bad these policies are below:

In [10]:
always_move_right(s) = 2;
always_move_left(s) = 1;
always_listen(s) = 3;

## Policy evaluation
We compute a policy’s value function (utility) using the `iterative_policy_evaluation` function, which evaluates the _immediate benefit_ of implementing the policy $\pi(s)$ and the _future benefit_ of looking ahead over states. 

* The `iterative_policy_evaluation(...)` function improves our estimate of the value (utility) of a policy $U^{\pi}(s)$ by iteration. The `iterative_policy_evaluation` function takes a `MyMDPProblemModel` instance, a policy function, and the maximum number of iterations to refine the value estimate.

```julia
function iterative_policy_evaluation(p::MyMDPProblemModel, policy::Function, k_max::Int)

    # grab stuff from the problem -
    𝒮, T, R, γ = p.𝒮, p.T, p.R, p.γ;

    # initialize U vector
    U = [0.0 for s ∈ 𝒮];

    # solve -
    for _ ∈ 1:k_max
        U = [lookahead(p, U, s, policy(s)) for s ∈ 𝒮]
    end

    # return U -
    return U;
end
```

The `lookahead(...)` function computes the immediate and future benefit. The `lookahead` function takes the `MyMDPProblemModel` instance, the value vector `U,` the state `s,` and the action `a` as arguments and returns the total value of the policy (immediate plus future benefit):

```julia
function lookahead(p::MyMDPProblemModel, U::Vector{Float64}, s::Int64, a::Int64)
    
    # get data from the problem
    𝒮, T, R, γ = p.𝒮, p.T, p.R, p.γ;
    
    # compute -
    return R[s,a] + γ*sum(T[s, s′,a]*U[i] for (i,s′) in enumerate(𝒮))
end
```

In [25]:
UPE = iterative_policy_evaluation(m, always_move_right, 50*length(states));

In [26]:
UPE

10-element Vector{Float64}:
    0.0
  -47.23239230918141
  -66.29107692516689
  -77.2959771954194
  -86.3885563869365
  -95.48177095161319
 -105.21331762767126
 -115.8405572304044
 -127.51213234222104
    0.0

## Estimate the Action-value or Q-function
The quantity being computed in the `lookahead(...)` function is called the _action-value_ or _Q-function_ $Q(s, a)$. From $Q(s,a)$ we can compute the value function $U(s)$ by selecting the action $a$ that maximizes the _Q-function_:
\begin{equation*}
U(s) = \underset{a\in\mathcal{A}}{\max}\,Q(s,a)
\end{equation*}
and the policy $\pi(s)$:
\begin{equation*}
\pi(s) = \underset{a\in\mathcal{A}}{\arg\max}\,Q(s,a)
\end{equation*}

### Implementation
* We compute the `Q-function` by calling the `Q(...)` function:

```julia
function Q(p::MyMDPProblemModel, U::Array{Float64,1})::Array{Float64,2}

    # grab stuff from the problem
    𝒮, T, R, γ = p.𝒮, p.T, p.R, p.γ;

    # initialize -
    Q_array = Array{Float64,2}(undef, length(𝒮), length(𝒜))

    for s ∈ 1:length(𝒮)
        for a ∈ 1:length(𝒜)
            Q_array[s,a] = R[s,a] + γ*sum([T[s, s′,a]*U[s′] for s′ in 𝒮]);
        end
    end

    return Q_array
end
```

* We estimate the best policy using the `policy(...)` function:

```julia
function policy(Q_array::Array{Float64,2})::Array{Int64,1}

    # get the dimension -
    (NR, _) = size(Q_array);

    # initialize some storage -
    π_array = Array{Int64,1}(undef, NR)
    for s ∈ 1:NR
        π_array[s] = argmax(Q_array[s,:]);
    end

    # return -
    return π_array;
end
```

In [13]:
QPE_array = Q(m, UPE)[2:end-1,:]

8×3 Matrix{Float64}:
 12.7512     8.25364  11.1137
 11.5841    10.5249   10.0049
 10.5213     9.55429   8.99527
  9.54818    8.654     8.07077
  8.63886    7.77503   7.20691
  7.7296     6.77497   6.34312
  6.62911    5.20109   5.29765
  4.72324  -98.4256    3.48708

In [14]:
best_policy = policy(QPE_array)

8-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1

### Greedy policy
Given a value (utility) function $U(s)$, we can construct a $\textit{greedy}$ policy $\pi(s)$ by selecting the action $a$ that maximizes the expected utility of the next state $s^{\prime}$:

$$
\begin{equation*}
\pi(s) = \underset{a\in\mathcal{A}}{\arg\max} \left(R(s,a)+\gamma\cdot\sum_{s^{\prime}\in\mathcal{S}}T(s^{\prime}\,\vert\, s,a)\cdot{U}(s^{\prime})\right)
\end{equation*}
$$

Let's explore this idea by constructing a greedy policy. First, we need to generate a value (utility) function $U(s)$, let's suppose it's random (uniform random between `safety` and `tiger`):

In [15]:
Uₒ = [rand() for s ∈ states];
Uₒ[safety] = 1000
Uₒ[tiger] = -1000

-1000

In [16]:
Uₒ

10-element Vector{Float64}:
  1000.0
     0.981573344299635
     0.14541689972201743
     0.9829701423368213
     0.8897969079855844
     0.8467567395575449
     0.574374688529497
     0.38799560238170094
     0.8899506739169001
 -1000.0

Next, let's create a `MyValueFunctionPolicy` instance, that takes the problem variable `m` and our random value vector $U_{\circ}(s)$ as arguments in its constructor:

In [17]:
my_greedy_value_policy = MyValueFunctionPolicy(m, Uₒ);

In [18]:
[1:10 my_greedy_value_policy.U]

10×2 Matrix{Float64}:
  1.0   1000.0
  2.0      0.981573
  3.0      0.145417
  4.0      0.98297
  5.0      0.889797
  6.0      0.846757
  7.0      0.574375
  8.0      0.387996
  9.0      0.889951
 10.0  -1000.0

Finally, we can estimate a policy using the `greedy(...)` function in combination with `a very Julia` way of calling the `greedy(...)` function:

```julia
function greedy(problem::MyMDPProblemModel, U::Array{Float64,1}, s::Int64)
    u, a = findmax(a->lookahead(problem, U, s, a), problem.𝒜);
    return (a=a, u=u)
end

(π::MyValueFunctionPolicy)(s::Int64) = greedy(π.problem, π.U, s).a;
```

In [19]:
my_greedy_value_policy(3)

2

In [20]:
QGREEDY_array = Q(m,Uₒ)[2:end-1,:]

8×3 Matrix{Float64}:
  722.535      237.604     -0.0675053
    0.932826     0.93349   -0.861854
    0.314936     0.668517  -0.0661784
    0.901471     0.83677   -0.154693
    0.770394     0.620569  -0.195581
    0.695463     0.477552  -0.454344
    0.620605     0.770504  -0.631404
 -237.224     -812.408     -0.154547

### Value Iteration
`Value iteration` iteratively computes the optimal value function $U^{*}(s)$ using the `Bellman backup` operation:

$$
\begin{equation*}
U_{k+1}(s) = \underset{a\in\mathcal{A}}{\max}\left(R(s,a) + \gamma\cdot\sum_{s^{\prime}\in\mathcal{S}}T(s^{\prime}\,\vert\,s,a)\cdot{U}_{k}(s^{\prime})\right)
\end{equation*}
$$

As $k\rightarrow\infty$ the value function $U_{k}(s)$ converges to the optimal value function $U^{\star}(s)$.The optimal policy $\pi^{\star}(s)$ can be extracted from the $Q(s,a)$-function (which is constructed from $U^{\star}(s)$):

$$
\begin{equation*}
Q^{\star}(s,a) = R(s,a) + \gamma\times{\text{sum}([T(s,s^{\prime},a)\times{U^{\star}}(s^{\prime})\,\,\text{for}\,s^{\prime} \in\mathcal{S}])}
\end{equation*}
$$

by selecting the action $a$ such that:

$$
\begin{equation*}
\pi^{\star}(s) = \underset{a\in\mathcal{A}}{\arg\max}\,Q^{\star}(s,a)
\end{equation*}
$$

#### Implementation
Let's explore value iteration by first constructing an instance of the `MyValueIterationModel` type, which takes the maximum number of iterations as a parameter. Save this in the `value_iteration_model` variable:

In [21]:
value_iteration_model = MyValueIterationModel(2);

Next, we call the `solve(...)` method by passing the `value_iteration_model` instance and our MDP model `m::MyMDPProblemModel` as arguments. The `solve(...)` method iteratively computes the value function $U^{\star}(s)$, by calling the `backup(...)` function, which in turn calls the `lookahead(...)` function:

```julia

function backup(problem::MyMDPProblemModel, U::Array{Float64,1}, s::Int64)
    return maximum(lookahead(problem, U, s, a) for a ∈ problem.𝒜);
end

function solve(model::MyValueIterationModel, problem::MyMDPProblemModel)::MyValueFunctionPolicy
    
    # initialize
    k_max = model.k_max
    U = [0.0 for _ ∈ problem.𝒮];

    # main loop -
    for _ ∈ 1:k_max
        U = [backup(problem, U, s) for s ∈ problem.𝒮];
    end

    return MyValueFunctionPolicy(problem, U);
end
```

The `solve(...)` method iteratively computes the optimal value function $U^{\star}(s)$ and returns it in an instance of the `MyValueFunctionPolicy` type. 

In [22]:
test = solve(value_iteration_model, m)

MyValueFunctionPolicy(MyMDPProblemModel([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3], [1.0 0.0 … 0.0 0.0; 0.75 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.25; 0.0 0.0 … 0.0 1.0;;; 1.0 0.0 … 0.0 0.0; 0.25 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.75; 0.0 0.0 … 0.0 1.0;;; 1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], [0.0 0.0 0.0; 10.0 0.0 -1.0; … ; 0.0 -100.0 -1.0; 0.0 0.0 0.0], 0.95), [0.0, 10.0, 7.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [23]:
test.U

10-element Vector{Float64}:
  0.0
 10.0
  7.125
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [24]:
my_π = Q(m, test.U)[2:end-1,:] |> policy

8-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1